In [9]:
# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
# actual vs predicted
models = [model for models in results.values() for model in models]

num_models = len(models)
num_cols = 2
num_rows = math.ceil(num_models / num_cols)

fig, axs = plt.subplots(num_rows, num_cols, figsize=(18, 6*num_rows), tight_layout=True)
axs = axs.flatten()

for i, model in enumerate(models):
    axs[i].scatter(y_test, model.y_pred, alpha=0.25)
    axs[i].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k', lw=1)
    axs[i].set_title(model.label)
    axs[i].set_xlabel('Actual Values')
    axs[i].set_ylabel('Predicted Values')

plt.suptitle("Actual vs. Predicted values by model")

plt.show(fig)

In [ ]:
# residuals
models = [model for models in results.values() for model in models]

num_models = len(models)
num_cols = 2
num_rows = math.ceil(num_models / num_cols)

fig, axs = plt.subplots(num_rows, num_cols, figsize=(18, 6*num_rows), tight_layout=True)

for i, model in enumerate(models):
    # Calculate residuals
    prediction_error = y_test[i] - model.y_pred

    # Extract the subplot for the current model
    row = i // num_cols
    col = i % num_cols

    # Plot the scatter plot on the specific subplot
    axs[row, col].scatter(model.y_pred, prediction_error, alpha=0.5)
    plt.axhline(y=0, color='r', linestyle='--')
    axs[row, col].set_title(model.label)
    axs[row, col].set_xlabel('Predicted Values')
    axs[row, col].set_ylabel('Prediction Errors')

plt.suptitle("Residual values by model")

plt.show(fig)

In [ ]:
# model evaluation
regressor_names = [result.label for results in results.values() for result in results]

mse_mean_cv_values = [result.mse_mean_cv for results in results.values() for result in results]
mse_test_values = [result.mse_test for results in results.values() for result in results]

bar_width = 0.35
index = range(len(regressor_names))

fig, ax = plt.subplots(tight_layout=True)
bar1 = ax.bar(index, mse_mean_cv_values, bar_width, label='MSE Mean CV')
bar2 = ax.bar([i + bar_width for i in index], mse_test_values, bar_width, label='MSE Test')

ax.set_xlabel('Regressor')
ax.set_ylabel('Mean Squared Error (MSE)')
ax.set_title('MSE Mean CV and MSE Test for Each Regressor')
ax.set_xticks([i + bar_width / 2 for i in index])
ax.set_xticklabels(regressor_names)
ax.legend()

plt.show(fig)

In [ ]:
# feature importances
fig, ax = plt.subplots(tight_layout=True)

# Bar plot for average feature importances
sns.barplot(x=feature_importances.index, y=feature_importances['average'], hue=feature_importances.index, legend=False)
plt.axhline(y=0, color='k', linestyle='--', linewidth=1)
ax.set_title('Average Feature Importances')
ax.set_xlabel('Features')
ax.set_ylabel('Average Importance')
ax.tick_params(axis='x', labelsize=8)

plt.show(fig)

In [ ]:
regressor_names = [result.label for results in results.values() for result in results]
training_times = [result.training_time for results in results.values() for result in results]

fig, ax = plt.subplots(tight_layout=True)
ax.bar(regressor_names, training_times, color='blue')
ax.set_xlabel('Regressor')
ax.set_ylabel('Training Time (s)')
ax.set_title('Training Time Comparison for Different Regressors')

plt.show(fig)